In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Description
I have already　submitted a prediction for missing values in [another notebook](https://www.kaggle.com/code/wasshoiwasshoi/tps-2022-jun-using-lightgbm-for-now/notebook).  
However, the prediction was based on learning with missing values.    

So　I try to fill in the missing values once with predictions, and I checked what would happen to the scores if I trained with those data.

# Libray import

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Data loading

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2022/data.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv')

In [ ]:
sub6 = pd.read_csv('../input/submission6/submission6.csv')

In [ ]:
temp = sub6['row-col'].str.split('-', expand=True)
sub6 = pd.concat([sub6, temp], axis =1).drop('row-col', axis=1)
sub6.rename(columns={0: 'row_No', 1: 'column_name'}, inplace=True)
sub6

In [ ]:
temp_pred_df = df.copy()
temp_pred_df.isnull().sum().sum()

In [ ]:
for i in range(len(temp_pred_df)):
    temp_row = int(sub6['row_No'][i])
    temp_col = sub6['column_name'][i]
    temp_pred_df[temp_col][temp_row] = sub6['value'][i]

In [ ]:
temp_pred_df.isnull().sum().sum()

In [ ]:
f1_columns = [c for c in df.columns if 'F_1' in c]
f2_columns = [c for c in df.columns if 'F_2' in c]
f3_columns = [c for c in df.columns if 'F_3' in c]
f4_columns = [c for c in df.columns if 'F_4' in c]

In [ ]:
submission_f1_df = pd.DataFrame(index=[], columns=['row-col', 'value'])
f1_column_results = []
for column in f1_columns:
    temp_test = df[df[column].isnull()]
    temp_train = temp_pred_df[~df[column].isnull()]
    result_df = temp_test[['row_id']]
    del temp_test[column]
    del temp_test['row_id']
    del temp_train['row_id']

    y = temp_train[column]
    X = temp_train.drop(column, axis = 1)

    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=100)
    train_set = lgb.Dataset(train_X, train_y)
    valid_set = lgb.Dataset(test_X, test_y)
    params = {
        "objective" : "regression",
        "metric" : "rmse"}
    print(f'\n\n {column}_calulate')
    model = lgb.train(
        params = params,
        train_set = train_set,
        valid_sets = [train_set, valid_set],
        num_boost_round = 4000)
    
    pred_test = model.predict(test_X)
    mse = mean_squared_error(test_y, pred_test)
    rmse = np.sqrt(mse)
    f1_column_results.append([column, rmse])
    
    pred = model.predict(temp_test)
    result_df['row-col'] = result_df['row_id'].astype(str) + f'-{column}'
    result_df['value']= pred
    result_df.reset_index(inplace=True, drop=True)
    submission_f1_df = pd.concat([submission_f1_df, result_df], join='inner')
submission_f1_df

In [ ]:
f1_column_results

In [ ]:
submission_f3_df = pd.DataFrame(index=[], columns=['row-col', 'value'])
f3_column_results = []
for column in f3_columns:
    temp_test = df[df[column].isnull()]
    temp_train = temp_pred_df[~df[column].isnull()]
    result_df = temp_test[['row_id']]
    del temp_test[column]
    del temp_test['row_id']
    del temp_train['row_id']

    y = temp_train[column]
    X = temp_train.drop(column, axis = 1)
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=100)
    train_set = lgb.Dataset(train_X, train_y)
    valid_set = lgb.Dataset(test_X, test_y)
    params = {
        "objective" : "regression",
        "metric" : "rmse"}
    print(f'\n\n {column}_calulate')
    model = lgb.train(
        params = params,
        train_set = train_set,
        valid_sets = [train_set, valid_set],
        num_boost_round = 4000)
    
    pred_test = model.predict(test_X)
    mse = mean_squared_error(test_y, pred_test)
    rmse = np.sqrt(mse)
    f3_column_results.append([column, rmse])
    
    pred = model.predict(temp_test)
    result_df['row-col'] = result_df['row_id'].astype(str) + f'-{column}'
    result_df['value']= pred
    result_df.reset_index(inplace=True, drop=True)
    submission_f3_df = pd.concat([submission_f3_df, result_df], join='inner')
submission_f3_df

In [ ]:
f3_column_results

In [ ]:
submission_f4_df = pd.DataFrame(index=[], columns=['row-col', 'value'])
f4_column_results = []
for column in f4_columns:
    temp_test = df[df[column].isnull()]
    temp_train = temp_pred_df[~df[column].isnull()]
    result_df = temp_test[['row_id']]
    del temp_test[column]
    del temp_test['row_id']
    del temp_train['row_id']

    y = temp_train[column]
    X = temp_train.drop(column, axis = 1)
    
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=100)
    train_set = lgb.Dataset(train_X, train_y)
    valid_set = lgb.Dataset(test_X, test_y)
    params = {
        "objective" : "regression",
        "metric" : "rmse"}
    print(f'\n\n {column}_calulate')
    model = lgb.train(
        params = params,
        train_set = train_set,
        valid_sets = [train_set, valid_set],
        num_boost_round = 4000)
    
    pred_test = model.predict(test_X)
    mse = mean_squared_error(test_y, pred_test)
    rmse = np.sqrt(mse)
    f4_column_results.append([column, rmse])
        
    pred = model.predict(temp_test)
    result_df['row-col'] = result_df['row_id'].astype(str) + f'-{column}'
    result_df['value']= pred
    result_df.reset_index(inplace=True, drop=True)
    submission_f4_df = pd.concat([submission_f4_df, result_df], join='inner')
submission_f4_df

In [ ]:
f4_column_results

In [ ]:
submission_all_df = pd.concat([submission_f1_df, submission_f3_df, submission_f4_df], join='inner')
submission_all_df

In [ ]:
del sub['value']

In [ ]:
submission = pd.merge(sub, submission_all_df, on='row-col', how='left')

In [ ]:
submission.to_csv("/kaggle/working/submission6_2.csv", index=False)

In [ ]:
column_results = []
column_results.append(f1_column_results)
column_results.append(f3_column_results)
column_results.append(f4_column_results)

In [ ]:
column_results

# Summary
I try to fill in the missing values once with predictions and I checked what would happen to the scores if I trained with those data.　As a result, I could not improve my score.